# Lab 5: Neural Networks for Text Classification

This lab introduces (deep) neural networks for text classification using Pytorch, and applies it to the datasets we previously used with naïve Bayes and logistic regression. Pytorch is a framework for machine learning with neural networks, which is widely used in fields such as Computer Vision and NLP. 

You may also find [Pytorch's tutorials](https://pytorch.org/tutorials/) useful to give more depth on different parts of the framework.

### Outcomes
* Be able to construct and train a neural network classifier in Pytorch.
* Understand how to use word embeddings as input to a neural network.
* Know how to compare classifier performance on a test set.

### Overview

We first format the data so it can be input to the neural network. Then we see how to construct a neural network with Pytorch, then train and test it. Finally, we introduce pretrained embeddings to the model.

# 1. Loading the Data

This section contains the same loader code as earlier labs, which loads the sentiment dataset from TweetEval.

In [ ]:
%matplotlib inline  
# do this first to make plots appear inside the notebook

In [ ]:
from datasets import load_dataset
from tqdm import tqdm

cache_dir = "./data_cache"

train_dataset = load_dataset(
    "tweet_eval",
    name="sentiment",
    split="train",
    ignore_verifications=True,
    cache_dir=cache_dir,
)

print(f"Training dataset with {len(train_dataset)} instances loaded")

dev_dataset = load_dataset(
    "tweet_eval",
    name="sentiment",
    split="validation",
    ignore_verifications=True,
    cache_dir=cache_dir,
)

print(f"Development/validation dataset with {len(dev_dataset)} instances loaded")

test_dataset = load_dataset(
    "tweet_eval",
    name="sentiment",
    split="test",
    ignore_verifications=True,
    cache_dir=cache_dir,
)

print(f"Test dataset with {len(test_dataset)} instances loaded")

# Put the data into lists ready for the next steps...
train_texts = []
train_labels = []
for i in tqdm(range(len(train_dataset))):
    train_texts.append(train_dataset[i]['text'])
    train_labels.append(train_dataset[i]['label'])

In [ ]:
import numpy as np 

# Show the training labels
np.unique(train_labels)

# 2. Preparing the Data
Now we put the dataset into a suitable format for a Pytorch NN classifier.

In [ ]:
import torch  # This imports the main Pytorch module -- Pytorch is the Python version of a library called 'Torch' so we just need to import torch.

%load_ext autoreload
%autoreload 2

As inputs to the Sklearn classifiers in week 3, we used CountVectorizer to extract a single vector representation for a *whole document*.
However, one motivation for using a neural network is that it can process the individual words in the sentence in order, and learn how to combine information from different tokens automatically. This means we don't need to convert the document to a fixed-length vector during the preprocessing phase.
Instead, as input to our neural network, we will pass in a sequence of tokens, where each token is represented by its *input_id*, which is its index into the vocabulary.

The first step is to compute the vocabulary. This can be done in various ways, but here we will stick with the familiar CountVectorizer method:

In [ ]:
# Use CountVectorizer to obtain a vocabulary
from sklearn.feature_extraction.text import CountVectorizer
from gensim.utils import tokenize

vectorizer = CountVectorizer(tokenizer=tokenize)  # use the Gensim tokenizer so we get consistency with the Gensim embeddings
vectorizer.fit(train_texts)

vocab = vectorizer.vocabulary_

In [ ]:
print(f'Size of vocabulary: {len(vocab)}')

Now, we have to map the tokens to their IDs -- their indexes in the vocabulary.

In [ ]:
# tokenize training set and convert to input IDs.
def encode_text(sample):
    tokens = tokenize(sample['text'])  # tokenise using Gensim, as in previous lab
    input_ids = []
    for token in tokens:
        if str.lower(token) in vocab:  # Skip words from the dev/test set that are not in the vocabulary.
            input_ids.append(vocab[str.lower(token)]+1) # +1 is needed because we reserve 0 as a special character
    sample['input_ids'] = input_ids  # seems not to like numpy arrays, so just use a list and convert to an array later
    return sample

train_dataset = train_dataset.map(encode_text)  # the datasets map function applies the function encode_text to all instances in the dataset
len(train_dataset)

Our neural network's input layer has a fixed size, so we need to somehow make all of our documents have the same number of tokens. We can do this by setting a fixed sequence length, then *padding* the short documents with a special token. Any documents that exceed the length will be truncated. Let's plot a histogram to understand the length distribution of the texts.

In [ ]:
import matplotlib.pyplot as plt

rv_l = [len(doc) for doc in train_dataset['input_ids']]
print('Mean of the document length: {}'.format(np.mean(rv_l)))
print('Median of the document length: {}'.format(np.median(rv_l)))
print('Maximum document length: {}'.format(np.max(rv_l)))

plt.hist(rv_l)

The code cell below in intended to pad any documents that are too short and truncate any that are too long, so that we obtain a set of sequences of equal length. 

**TODO 2.1:** Complete the padding code below to insert 0s at the start of any sequences that are too short, and to truncate any sequences that are too long.

In [ ]:
sequence_length = 40  # truncate all docs longer than this. Pad all docs shorter than this.

def pad_text(sample):
    ###WRITE YOUR OWN CODE HERE

    ##########
    return sample

train_dataset = train_dataset.map(pad_text)

We now have our data in the right format. When training, the neural network will process the data in randomly-chosen mini-batches, rather than all at once.
To enable this, we wrap our dataset in a DataLoader, which allows the network to select batches of data:

DataLoader: https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64

# convert from the Huggingface format to a TensorDataset so we can use the mini-batch sampling functionality
def convert_to_data_loader(dataset, num_classes):
    # convert from list to tensor
    input_tensor = torch.from_numpy(np.array(dataset['input_ids']))
    label_tensor = torch.from_numpy(np.array(dataset['label'])).long()
    tensor_dataset = TensorDataset(input_tensor, label_tensor)
    loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=True)

    return loader

num_classes = len(np.unique(train_labels))   # number of possible labels in the sentiment analysis task

train_loader = convert_to_data_loader(train_dataset, num_classes)

Let's process the development and test set as well.

In [ ]:
dev_dataset = dev_dataset.map(encode_text)
dev_dataset = dev_dataset.map(pad_text)
dev_loader = convert_to_data_loader(dev_dataset, num_classes)

test_dataset = test_dataset.map(encode_text)
test_dataset = test_dataset.map(pad_text)
test_loader = convert_to_data_loader(test_dataset, num_classes)

# 3. Constructing the Network

We will build a NN with three different layers for sentiment classification.

### Embedding layer
In the embedding layer, the network will create its own embeddings for the index with a given embedding dimension.
The module `nn.Embedding()` creates a simple lookup table that stores embeddings of words in a fixed dictionary with fixed size.
This module is often used to store word embeddings and retrieve them using indices.
The module's input is a list of indices, and the output is the corresponding word embeddings.

[Documentation for Embedding Class](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

### Fully-connected layer
Fully-connected layers in a neural network are those layers where all the inputs from the previous layer are connected to every unit of the fully-connected layer.
Here we will use fully-connected layers for the hidden layer and output layer. In Pytorch this kind of layer is implemented by the 'Linear' class. The name 'linear' is used because the nonlinear part is provided by the activation functions, which act like another layer in the network.

https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

## Activation functions
In Pytorch, the activation function is not included in the Linear class (or other kinds of neural network layer). An example of an activation function is ReLU, which is commonly used in the hidden layers of a neural network:

https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

In Pytorch, we construct a neural network by connecting up the output of each component to the input of the next, thereby creating a computation graph.
To complete a fully-connected hidden layer, we connect the ouput of a Linear layer to the input of a ReLU activation function, thereby creating a nonlinear function.

**TODO 3.1** Complete the constructor for a NN with three layers by adding the missing dimensions.

**TODO 3.2** Complete the forward function that maps the input data to an output by adding the missing line.

In [ ]:
from torch import nn

class FFTextClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, hidden_size, num_classes):
        super(FFTextClassifier, self).__init__()

        self.embedding_size = embedding_size

        ### COMPLETE THE CODE HERE: WRITE IN THE MISSING ARGUMENTS SPECIFYING THE DIMENSIONS OF EACH LAYER

        # Here we just need to construct the components of our network. We don't need to connect them together yet.
        self.embedding_layer = nn.Embedding(???) # embedding layer
        self.hidden_layer = nn.Linear(???) # Hidden layer
        self.activation = nn.ReLU(???) # Hidden layer
        self.output_layer = nn.Linear(???) # Full connection layer

        ##########
        
        
    def forward (self, input_words):
        # Input dimensions are:  (batch_size, seq_length)
        embedded_words = self.embedding_layer(input_words)  # (batch_size, seq_length, embedding_size)

        # flatten the sequence of embedding vectors for each document into a single vector.
        embedded_words = embedded_words.reshape(embedded_words.shape[0], sequence_length*self.embedding_size)  # batch_size, seq_length*embedding_size

        ### ADD THE MISSING LINES HERE

        ########

        output = self.output_layer(h)                      # (batch_size, num_classes)

        # Notice we haven't applied a softmax activation to the output layer -- it's not required by Pytorch's loss function.

        return output


**TODO 3.3** Create a NN with the FFTextClassifier class we wrote.

**Hint:** `model = FFTextClassifier(...)`

In [ ]:
vocab_size = len(vectorizer.vocabulary_) + 1
embedding_size = 25  # number of dimensions for embeddings
hidden_size = 32 # number of hidden units

###WRITE YOUR OWN CODE HERE


After desigining our network, we need to create a training function to calculate the loss for each input and perform backpropagation to optimise the network.
During training, the weights of all the layers will be updated.

We build a training function to train the NN over a fixed number of epochs (an epoch is one iteration over the whole training dataset).
The function also prints the performance of both training and development/validation set after each epoch. There are some high-level wrapper libraries that do this stuff for you, but when learning about neural networks, it's useful to see what's going on inside.

**TODO 3.4** Complete the code below to compute the validation accuracy and loss after each training epoch.

In [ ]:
def train_nn(num_epochs, model, train_dataloader, dev_dataloader, loss_fn, optimizer):
    
    for e in range(num_epochs):
        # Track performance on the training set as we are learning...
        total_correct = 0
        total_trained = 0
        train_losses = []

        model.train()  # Put the model in training mode.

        for i, (batch_input_ids, batch_labels) in enumerate(train_dataloader):
            # Iterate over each batch of data
            # print(f'batch no. = {i}')

            optimizer.zero_grad()  # Reset the optimizer

            # Use the model to perform forward inference on the input data.
            # This will run the forward() function.
            output = model(batch_input_ids)

            # Compute the loss for the current batch of data
            batch_loss = loss_fn(output, batch_labels)

            # Perform back propagation to compute the gradients with respect to each weight
            batch_loss.backward()

            # Update the weights using the compute gradients
            optimizer.step()

            # Record the loss from this sample to keep track of progress.
            train_losses.append(batch_loss.item())

            # Count correct labels so we can compute accuracy on the training set
            predicted_labels = output.argmax(1)
            total_correct += (predicted_labels == batch_labels).sum().item()
            total_trained += batch_labels.size(0)

        train_accuracy = total_correct/total_trained*100

        print("Epoch: {}/{}".format((e+1), num_epochs),
              "Training Loss: {:.4f}".format(np.mean(train_losses)),
              "Training Accuracy: {:.4f}%".format(train_accuracy))

        model.eval()  # Switch model to evaluation mode
        total_correct = 0
        total_trained = 0
        dev_losses = []

        for dev_input_ids, dev_labels in dev_dataloader:
            ###WRITE YOUR OWN CODE HERE

            
            #######

            # Save the loss on the dev set
            dev_losses.append(dev_loss.item())

            # Count the number of correct predictions
            predicted_labels = dev_output.argmax(1)
            total_correct += (predicted_labels == dev_labels).sum().item()
            total_trained += dev_labels.size(0)
            
        dev_accuracy = total_correct/total_trained*100
        
        print("Epoch: {}/{}".format((e+1), num_epochs),
              "Validation Loss: {:.4f}".format(np.mean(dev_losses)),
              "Validation Accuracy: {:.4f}%".format(dev_accuracy))
    return model


The last step before we start training is defining the loss function and optimizer.

Here we use cross-entropy loss and the Adam optimizer (it tends to find a better solution in a small number of iterations than SGD).
The module `nn.CrossEntropyLoss()` combines `LogSoftmax` and `NLLLoss` in one single class so that we don't have to implement the softmax layer within the forward() method.

Cross Entropy Loss: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

Optimization: https://pytorch.org/docs/stable/optim.html

**TODO 3.4** Finally, train the network for 10 epochs!

In [ ]:
from torch import optim

learning_rate = 0.0005

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(ff_classifier_model.parameters(), lr=learning_rate)

###WRITE YOUR OWN CODE HERE



**TODO 3.5:** Evaluate the model on test set using the function below. Complete the code to count the correct classifications.

In [ ]:
def test_nn(trained_model, test_loader, loss_fn):

    trained_model.eval()

    test_losses = []
    correct = 0  # count the number of correct classification labels

    for inputs, labels in test_loader:
        test_output = trained_model(inputs)
        loss = loss_fn(test_output, labels)
        test_losses.append(loss.item())
        predicted_labels = test_output.argmax(1)

        ###WRITE YOUR OWN CODE HERE

        
        ######

    accuracy = correct/len(test_loader.dataset)*100
    print("Test Accuracy: {:.2f}%".format(accuracy))
    # print(predicted)

test_nn(trained_model, test_loader, loss_fn)

# 4. Pretrained Embeddings

Now let's use pretrained word embeddings as inputs instead of learning them from scratch during training.
Here, we will use a pretrained embedding matrix to initialise the embedding layer, which will then be updated during training.

The class below extends the FFTextClassifier class. This means that it inherits all of its functionality, but we now overwrite the constructor (the `__init__` method).
This way, we don't need to define the forward function again, as it will be the same as before.

**TODO 4.1** As before, complete the arguments below to set the dimensions of the neural network layers.

In [ ]:
import gensim.downloader

class FFTextClassifierWithEmbeddings(FFTextClassifier):

    def __init__(self, hidden_size, num_classes):
        super(FFTextClassifier, self).__init__()

        # download pretrained embeddings as the intial values for out embedding layer.
        embedding_matrix = gensim.downloader.load('glove-twitter-25')  # 'word2vec-google-news-300')
        self.embedding_size = embedding_matrix.vectors.shape[1]
        embedding_matrix_reordered = torch.zeros((vocab_size, self.embedding_size))
        for word in vocab:
            if word in embedding_matrix:
                embedding_matrix_reordered[vocab[word]] = torch.from_numpy(embedding_matrix[word])

        # Here we just need to construct the components of our network. We don't need to connect them together yet.
        self.embedding_layer = nn.Embedding.from_pretrained(embedding_matrix_reordered, freeze=False) # embedding layer

        ### COMPLETE THE ARGUMENTS TO SPECIFY THE DIMENSIONS OF THE LAYERS
        self.hidden_layer = nn.Linear(???) # Hidden layer
        self.activation = nn.ReLU(???) # Hidden layer
        self.output_layer = nn.Linear(???) # Full connection layer

**TODO 4.2** Using the above class, construct, train and test the classifier with pretrained embeddings. You will need to create a new optimizer object.

In [ ]:
### WRITE YOUR OWN CODE BELOW

